In [6]:
from src.data.db import connect_to_db


conn = connect_to_db()
cursor = conn.cursor()

In [2]:
cursor.execute("""
            SELECT i.main_category, i.sub_category, i.description, c.body
            FROM jira_issues i, jira_comments c
            WHERE i.issue_key = c.issue_key
               """)

results = cursor.fetchall()
cursor.close()
conn.close()

In [14]:
from src.utils.constants import CATEGORIES
from collections import defaultdict

ticket_raw = defaultdict(str)
    

In [15]:
for main, sub, desc, comm in results:
    ticket_raw[f"{main}-{sub}"] += (f"\n Description: {desc} \n Comments: {comm} \n ***END OF TICKET***")

In [19]:
for key in ticket_raw.keys():
    with open(f"{key}.txt", "a") as f:
        f.write(ticket_raw[key])

In [23]:
for key in ticket_raw.keys():
    print(key)

delivery-issues-damaged-goods
order-modifications-item-removals
emergency-situations-weather-related-disruptions
delivery-issues-missed-deliveries
order-modifications-item-additions
other-miscellaneous
technical-support-email-&-communication-issues
order-modifications-quantity-adjustments
delivery-issues-delivery-confirmation-issues
order-cancellations-rescheduled-orders
delivery-issues-late-deliveries
grant-&-billing-issues-grant-fund-usage
pickup-scheduling-&-rescheduling-rescheduling-pickup
product-availability-&-substitutions-special-item-requests
technical-support-website-&-ordering-system-errors
pickup-scheduling-&-rescheduling-new-pickup-requests
training-&-account-access-new-user-account-requests
product-availability-&-substitutions-stock-availability-inquiries
pickup-scheduling-&-rescheduling-missed-pickups
training-&-account-access-missed-training-sessions
training-&-account-access-training-signups
grant-&-billing-issues-payment-&-credit-issues
grant-&-billing-issues-billing-

In [ ]:
CATEGORIES = {'order-modifications': ['item-additions',
  'item-removals',
  'quantity-adjustments',
  'late-modification-requests'],
 'order-cancellations': ['standard-cancellations',
  'urgent-cancellations',
  'rescheduled-orders'],
 'delivery-issues': ['late-deliveries',
  'missed-deliveries',
  'incomplete-deliveries',
  'damaged-goods',
  'delivery-confirmation-issues'],
 'pickup-scheduling-&-rescheduling': ['new-pickup-requests',
  'rescheduling-pickup',
  'missed-pickups',
  'pickup-policy-clarifications'],
 'product-availability-&-substitutions': ['stock-availability-inquiries',
  'out-of-stock-notifications',
  'product-substitution-requests',
  'special-item-requests'],
 'grant-&-billing-issues': ['grant-fund-usage',
  'incorrect-grant-deduction',
  'billing-discrepancies',
  'payment-&-credit-issues'],
 'training-&-account-access': ['training-signups',
  'missed-training-sessions',
  'login-issues',
  'new-user-account-requests'],
 'emergency-situations': ['weather-related-disruptions',
  'personal/organization-emergencies',
  'food-safety-concerns',
  'unexpected-facility-closures'],
 'special-requests': ['educational-materials',
  'large-event-orders',
  'holiday-&-seasonal-adjustments'],
 'technical-support': ['website-&-ordering-system-errors',
  'email-&-communication-issues',
  'data-entry-mistakes',
  'general-it-assistance'],
 'other': ['miscellaneous']}

In [70]:
from src.config import PROCESSED_DATA_DIR
from psycopg2.extras import execute_values
from datetime import datetime

with open(PROCESSED_DATA_DIR / "kb"/ "KB_order-modifications-item-removals.txt", "r") as f:
    kb = f.read()
    execute_values(cursor,
                   """
INSERT INTO knowledge_base 
(        main_category,
        sub_category,
        kb,
        created_at,
        updated_at)
VALUES %s
                   """,
[('order-modifications',
  'item-removals', 
  kb,
  datetime.now(),
  datetime.now())])

In [71]:
conn.commit()

Hey, I want you to help me create a detailed Knowledge Base for Capital Area Food Bank's support system. A small example file for KB is attached as well to help you structure the KB
 This Knowledge base will be used by a LLM backed chatbot to help answer partners questions, concerns and queries and will be given as context to the LLM to get relevant knowledge about that particular category of issues. Make it as detailed as possible to help the LLM output good responses
You will be given a txt file containing ticket descriptions and their comments for a specific ticket category. The ticket category will be the name of the txt file. You need to create a Knowledge base based upon this txt file. Include FAQs, troubleshooting guides, and best practices relevant to CAFB Partners as well as any other relevant details that will equip the LLM to best help the partners 
Generate txt that can be directly copied to a txt file

DO NOT HAVE ANY MISINFORMATION IN THE KB. For example, the LLM won't specifically know about delivery dates, so it shouldnt lie